In [307]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
from datetime import datetime
from dateutil import relativedelta

## pdf2text

In [38]:
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io
from datetime import datetime

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as fh:
        # iterate over all pages of PDF document
        for page in PDFPage.get_pages(fh, caching=True, check_extractable=True):
            # creating a resoure manager
            resource_manager = PDFResourceManager()
            
            # create a file handle
            fake_file_handle = io.StringIO()
            
            # creating a text converter object
            converter = TextConverter(
                                resource_manager, 
                                fake_file_handle, 
                                codec='utf-8', 
                                laparams=LAParams()
                        )

            # creating a page interpreter
            page_interpreter = PDFPageInterpreter(
                                resource_manager, 
                                converter
                            )

            # process current page
            page_interpreter.process_page(page)
            
            # extract text
            text = fake_file_handle.getvalue()
            yield text

            # close open handles
            converter.close()
            fake_file_handle.close()

In [262]:
# calling above function and extracting text
#file_path = u'/Users/eileen/DataChallenges/FORKAIA/RESUME/pyresparser-master/OmkarResume.pdf'
# file_path = '/Users/eileen/DataChallenges/FORKAIA/RESUME/pyresparser-master/sampleCV.pdf'
file_path = '/Users/eileen/Desktop/Xinxin Gu - Data Science - FT.pdf'
# file_path = '/Users/eileen/DataChallenges/FORKAIA/RESUME/sample2.pdf'
text = ''
for page in extract_text_from_pdf(file_path):
    text += ' ' + page

In [263]:
text

' Xinxin Gu  \n\n \n \n \n \n \n \n\nxinxingu7@outlook.com | (585)-754-2812 | 10011 Stonelake Blvd, Austin, TX, 78759 \n\nwww.linkedin.com/in/xinxin7 | https://github.com/xinxin7 \n\n \nEDUCATION \nUniversity of Rochester                                                                                                                                                   Rochester, NY                                                  \nMaster of Data Science (30% Merit based Scholarship, GHC19’ Scholar)                                               Aug 2018 – Dec 2019 \nCoursework: Data Mining, Machine Learning, Database System, Algorithms, Data Structure, Marketing Analysis \nSouthwest Jiaotong University                                                                                                                                  Chengdu, China                                              \nBachelor of Engineering in Remote Sense Science and Technology (GPA: 3.72/4.0)                      

## extract name

In [264]:
import spacy
from spacy.matcher import Matcher
from collections import Counter

nlp = spacy.load('en_core_web_sm')
# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)
    # First name and Last name are always Proper Nouns
#    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN', 'OP': '?'}, {'POS': 'PROPN'}]
    matcher.add('NAME', None, pattern)
    matches = matcher(nlp_text)
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

In [265]:
name = extract_name(text)
name

'Xinxin Gu'

## extract phone numer

In [266]:
import re
def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)
    
    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [267]:
extract_mobile_number(text)

'5857542812'

## Extract Email

In [268]:
import re
def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [269]:
extract_email(text)

'xinxingu7@outlook.com'

## Extract Education

In [270]:
import re
import spacy
from nltk.corpus import stopwords

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# Grad all general stop words
STOPWORDS = set(stopwords.words('english'))

# Education Degrees
EDUCATION = [
            'BE','B.E.', 'B.E', 'BS', 'B.S', 
            'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
            'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
            'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII', 'MASTER', 'BACHELOR'
        ]

def extract_education(resume_text):
    nlp_text = nlp(resume_text)

    # Sentence Tokenizer
    nlp_text = [sent.string.strip() for sent in nlp_text.sents]

    edu = {}
    # Extract education degree
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            # Replace all special symbols
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index + 1]

    # Extract year
    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

In [271]:
extract_education(text)

[('Master', '2018'), ('Bachelor', '2013')]

## 6.  extract sections

In [314]:
RESUME_SECTIONS_GRAD = [
                    'accomplishments',
                    'experience',
                    'education',
                    'interests',
                    'projects',
                    'professional experience',
                    'publications',
                    'skills',
                    'certifications',
                    'objective',
                    'career objective',
                    'summary',
                    'leadership',
                    'college name'
                ]

In [315]:
def extract_entity_sections_grad(text):
    '''
    Helper function to extract all the raw text from sections of
    resume specifically for graduates and undergraduates

    :param text: Raw text of resume
    :return: dictionary of entities
    '''
    text_split = [i.strip() for i in text.split('\n')]
    # sections_in_resume = [i for i in text_split if i.lower() in sections]
    entities = {}
    key = False
    for phrase in text_split:
        if len(phrase) == 1:
            p_key = phrase
        else:
            p_key = set(phrase.lower().split()) & set(RESUME_SECTIONS_GRAD)
        try:
            p_key = list(p_key)[0]
        except IndexError:
            pass
        if p_key in RESUME_SECTIONS_GRAD:
            entities[p_key] = []
            key = p_key
        elif key and phrase.strip():
            entities[key].append(phrase)

    # entity_key = False
    # for entity in entities.keys():
    #     sub_entities = {}
    #     for entry in entities[entity]:
    #         if u'\u2022' not in entry:
    #             sub_entities[entry] = []
    #             entity_key = entry
    #         elif entity_key:
    #             sub_entities[entity_key].append(entry)
    #     entities[entity] = sub_entities

    # pprint.pprint(entities)

    # make entities that are not found None
    # for entity in cs.RESUME_SECTIONS:
    #     if entity not in entities.keys():
    #         entities[entity] = None
    return entities

In [316]:
sections = extract_entity_sections_grad(text)

In [317]:
for k, v in sections.items():
    print(k, v)
    print('*************************')

education ['University of Rochester                                                                                                                                                   Rochester, NY', 'Master of Data Science (30% Merit based Scholarship, GHC19’ Scholar)                                               Aug 2018 – Dec 2019', 'Coursework: Data Mining, Machine Learning, Database System, Algorithms, Data Structure, Marketing Analysis', 'Southwest Jiaotong University                                                                                                                                  Chengdu, China', 'Bachelor of Engineering in Remote Sense Science and Technology (GPA: 3.72/4.0)                              Sep 2013 – Jul 2017', 'Coursework: Computer Vision, Spatial Data Visualization and Modeling, Digital Image Processing']
*************************
skills ['Programming: Python, R, MySQL, T-SQL, Spark, HTML', 'Marketing Analysis: Descriptive Analysis, Casual Analysis, P

In [289]:
def extract_entities_wih_custom_model(custom_nlp_text):
    '''
    Helper function to extract different entities with custom
    trained model using SpaCy's NER

    :param custom_nlp_text: object of `spacy.tokens.doc.Doc`
    :return: dictionary of entities
    '''
    entities = {}
    for ent in custom_nlp_text.ents:
        if ent.label_ not in entities.keys():
            entities[ent.label_] = [ent.text]
        else:
            entities[ent.label_].append(ent.text)
    for key in entities.keys():
        entities[key] = list(set(entities[key]))
    return entities

In [294]:
nlp_text = nlp(text)
cus_sections = extract_entities_wih_custom_model(nlp_text)

In [295]:
for k, v in cus_sections.items():
    print(k, v)
    print('*************************')

PERSON ['Data Mining', '3.72/4.0', 'Xinxin Gu', 'Tableau', 'Robert Bosch LLC']
*************************
DATE ['Sep 2013', 'May 2019', '16 days', '10011', 'May 2018', 'June 2019', '78759', 'past 30 years', '2018', '2019']
*************************
GPE ['Python', 'Austin', 'Helpdesk Tickets', 'China', 'Built', 'Seaborn', 'Spark', 'Blvd', 'NY']
*************************
ORG ['Designed ETL', 'Collaborative Filtering \n\n', 'Tickets', 'PROJECTS', 'Healthcare Information Systems', 'TF-IDF', 'Spatial Data Visualization', 'GPA', 'Medical Center of University of Rochester', 'AWS', 'NLP', 'Achieved', 'SCI', 'T-SQL', 'ETL', 'SQL', 'Deloitte Consulting co.', 'Applied Random Forest', 'Python  \n', 'TX', 'Parameters', 'SPSS', 'Python', 'NLTK', 'Created', 'TensorFlow', 'EMR', 'LDA', 'BOW', 'ALS', 'Bosch', 'University of Rochester                                                                                                                                                   ', 'Tableau', 'XGBoost']
*

## 7. experience months

In [299]:
experience_list = sections['experience']

In [309]:
def get_number_of_months_from_dates(date1, date2):
    '''
    Helper function to extract total months of experience from a resume

    :param date1: Starting date
    :param date2: Ending date
    :return: months of experience from date1 to date2
    '''
    if date2.lower() == 'present':
        date2 = datetime.now().strftime('%b %Y')
    try:
        if len(date1.split()[0]) > 3:
            date1 = date1.split()
            date1 = date1[0][:3] + ' ' + date1[1]
        if len(date2.split()[0]) > 3:
            date2 = date2.split()
            date2 = date2[0][:3] + ' ' + date2[1]
    except IndexError:
        return 0
    try:
        date1 = datetime.strptime(str(date1), '%b %Y')
        date2 = datetime.strptime(str(date2), '%b %Y')
        months_of_experience = relativedelta.relativedelta(date2, date1)
        months_of_experience = (months_of_experience.years
                                * 12 + months_of_experience.months)
    except ValueError:
        return 0
    return months_of_experience

def get_total_experience(experience_list):
    '''
    Wrapper function to extract total months of experience from a resume

    :param experience_list: list of experience text extracted
    :return: total months of experience
    '''
    exp_ = []
    for line in experience_list:
        experience = re.search(
            r'(?P<fmonth>\w+.\d+)\s*(\D|to)\s*(?P<smonth>\w+.\d+|present)',
            line,
            re.I
        )
        if experience:
            exp_.append(experience.groups())
    total_exp = sum(
        [get_number_of_months_from_dates(i[0], i[2]) for i in exp_]
    )
    total_experience_in_months = total_exp
    return total_experience_in_months

In [310]:
get_total_experience(experience_list)

16